In [24]:
import os
#探索活动	探索/探究/研究/研发/科研，搜寻/搜索，变化/变异，风险/冒险，实验，创新/设计，灵活，发现
#开发活动	利用/应用/开发，提炼/精炼/精益，稳定，选择，生产/投产，效率/成本，改进/改良，执行/实施
freq_words = ["探索","探究","研究","研发","科研","搜寻","搜索","变化","变异","风险","冒险","实验","创新","设计","灵活","发现",
              "利用","应用","开发","提炼","精炼","精益","稳定","选择","生产","投产","效率","成本","改进","改良","执行","实施"]
              


def find_files(base_dir):
    return [base_dir+f for f in os.listdir(base_dir) if f.endswith('.txt')]

base_dir='../scraping/content/'
txt_files = find_files(base_dir)
print(len(txt_files))
print(txt_files[:4])

18673
['../scraping/content/300354_2015.txt', '../scraping/content/603041_2017.txt', '../scraping/content/002426_2013.txt', '../scraping/content/300381_2016.txt']


In [25]:

# read file to one line

def read_content(file):
    try:   
        lines=[];
        with open(file, 'r', encoding='utf-8') as f:
            for line in f.readlines():                          #依次读取每行  
                line = line.strip()
                if len(line)>0:
                    lines.append(line)

        content="".join(lines)
        return content
    except:
        pass
    
    return ""


# test the function
file = '../scraping/content/300354_2015.txt'
content=read_content(file)
print(content[:256])


江苏东华测试技术股份有限公司 2015 年年度报告全文江苏东华测试技术股份有限公司2015 年年度报告2016 年 03 月1江苏东华测试技术股份有限公司 2015 年年度报告全文第一节  重要提示、目录和释义本公司董事会、监事会及董事、监事、高级管理人员保证年度报告内容的真实、准确、完整，不存在虚假记载、误导性陈述或重大遗漏，并承担个别和连带的法律责任。公司负责人刘士钢、主管会计工作负责人范敏及会计机构负责人(会计主管人员)范敏声明：保证年度报告中财务报告的真实、准确、完整。所有董事均已出席了审议本报告的


In [26]:

def word_freq(content,freq_words):
    freq_words_dict = {k:0 for k in freq_words}
    start = 0
    while start < len(content)-3:
        word = content[start:start+2] # 取两个词
        if word in freq_words_dict: freq_words_dict[word]+=1
        start+=2
    return freq_words_dict

# test the function
freq_words_dict= word_freq(content,freq_words)
print(freq_words_dict)

{'探索': 1, '探究': 0, '研究': 19, '研发': 31, '科研': 2, '搜寻': 0, '搜索': 0, '变化': 7, '变异': 0, '风险': 21, '冒险': 0, '实验': 5, '创新': 13, '设计': 6, '灵活': 5, '发现': 5, '利用': 4, '应用': 15, '开发': 40, '提炼': 0, '精炼': 0, '精益': 0, '稳定': 4, '选择': 4, '生产': 41, '投产': 1, '效率': 3, '成本': 54, '改进': 2, '改良': 0, '执行': 3, '实施': 27}


In [27]:
from tqdm import tqdm
import pandas as pd
records=[]
# for file in tqdm(txt_files[:4]):
for file in tqdm(txt_files[:400]):
    content=read_content(file)
    freq_words_dict= word_freq(content,freq_words)
    code,year = os.path.basename(file).replace('.txt','').split('_')
    freq_words_dict['Symbol']=code
    freq_words_dict['EndDate']=year+'-12-31'
    records.append(freq_words_dict)
    

df = pd.DataFrame(records)
df.head()




100%|██████████| 400/400 [00:13<00:00, 30.21it/s]


,EndDate,Symbol,冒险,创新,利用,发现,变化,变异,实施,实验,...,生产,研发,研究,科研,稳定,精炼,精益,设计,选择,风险
0,2015-12-31,300354,0,13,4,5,7,0,27,5,...,41,31,19,2,4,0,0,6,4,21
1,2017-12-31,603041,0,8,4,6,13,0,23,0,...,60,22,12,1,38,0,1,7,1,42
2,2013-12-31,002426,0,2,1,1,2,0,0,0,...,2,3,0,0,0,0,0,0,0,2
3,2016-12-31,300381,0,9,6,8,6,0,22,4,...,115,32,16,6,18,0,0,1,5,36
4,2012-12-31,002285,0,0,3,1,5,0,2,0,...,0,0,1,0,0,0,0,2,1,5


In [28]:
cols = df.columns.tolist()
cols.remove('Symbol')
cols.insert(0,'Symbol')
df = df[cols]
df.head()

,Symbol,EndDate,冒险,创新,利用,发现,变化,变异,实施,实验,...,生产,研发,研究,科研,稳定,精炼,精益,设计,选择,风险
0,300354,2015-12-31,0,13,4,5,7,0,27,5,...,41,31,19,2,4,0,0,6,4,21
1,603041,2017-12-31,0,8,4,6,13,0,23,0,...,60,22,12,1,38,0,1,7,1,42
2,002426,2013-12-31,0,2,1,1,2,0,0,0,...,2,3,0,0,0,0,0,0,0,2
3,300381,2016-12-31,0,9,6,8,6,0,22,4,...,115,32,16,6,18,0,0,1,5,36
4,002285,2012-12-31,0,0,3,1,5,0,2,0,...,0,0,1,0,0,0,0,2,1,5


In [29]:
df2 = pd.read_excel('./研究数据0410.xlsx',index_col=None)
df3 = df2.merge(df,how='left',on=['Symbol','EndDate'])


df3.to_excel('研究数据0421_words.xlsx')

,Symbol,EndDate,Stknme,Listexg,Nindcd,Area,Listdt,Ucpd,Ucsvr,Nshld,...,生产,研发,研究,科研,稳定,精炼,精益,设计,选择,风险
0,没有单位',没有单位',没有单位',没有单位',没有单位',没有单位',没有单位',没有单位',没有单位',人',...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,证券代码',统计截止日期',证券简称',上市交易所',行业代码',地区',上市日期',实际控制人是否担任董事长或总经理',实际控制人拥有上市公司控制权比例(%)',股东总数',...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,000001,2017-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,000001,2016-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,000001,2015-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
